# Momentum Portfolio

For this momentum portfolio project, I will be using CRSP stock data to replicate the momentum decile returns in the 2016 paper, "Momentum Crashes" by Daniel and Moskowitz. The objective of this project is to determine the average annual returns for 10 different momentum portfolio deciles. Each decile categorizes every stock based on the total return for the last year of each stock. The bottom decile stocks provided the worst returns for the past year and the top deciles were the top performers of the past year. We also create the momentum portfolio outlined by Ken French (KRF) and compare it's returns to that of the Daniel and Moskowitz (DM) portfolio.

The risk free rate is from the French Fama numbers (not CRSP), which I download below. I combine the dlret and ret data for the stocks and create a new 'ret' column to encompass all returns for the stocks. I filter the stocks to only use the 10 and 11 sharecodes and the 1, 2 and 3 exchanges, per the French Fama website. 

Below is a list of the libraries I used for this project.

In [1]:
import wrds
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import pandas_datareader
import datetime
import numpy as np
import matplotlib.pyplot as plt

Below I connected the script to the “WRDS” servers so I could access the data for the stocks.

In [2]:
id = 'johncrowe'
conn = wrds.Connection(id)

# Download monthly crsp stock data
a = conn.raw_sql("""
                    select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                    a.ret, a.retx, a.shrout, a.prc, a.cfacshr, a.cfacpr
                    from crspq.msf as a 
                    left join crsp.msenames as b
                    on a.permno=b.permno
                    and b.namedt<=a.date
                    and a.date<=b.nameendt
                    where a.date between '01/01/1900' and '12/31/2023'
                    """)

# Download monthly crsp delisted stock data
b = conn.raw_sql("""
                    select permno, dlret, dlstdt, dlstcd
                    from crspq.msedelist
                    """)

c = conn.get_table(library='ff', table='factors_monthly')

conn.close()

Enter your WRDS username [John]:johncrowe
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


Below we create new dataframes so that we do not have to reconnect to the SQL servers to rerun our code.

In [18]:
crsp_raw = a.copy()
dlret_raw = b.copy()
french = c.copy()

Below we are importing data for the monthly risk free rate.

In [19]:
# Rearrange french data
french = french[['date','mktrf','smb','hml','rf']]
french['mkt'] = french['mktrf'] + french['rf']
french['date'] = pd.to_datetime(french['date']) + MonthEnd(0)

# Change columns to floats
french['mktrf'] = french['mktrf'].astype(float)
french['smb'] = french['smb'].astype(float)
french['hml'] = french['hml'].astype(float)
french['rf'] = french['rf'].astype(float)

# Create year and month columns for french data
french['year'] = french['date'].dt.year
french['month'] = french['date'].dt.month

Here we use some data cleaning and rearraning. First we arrange by date and permno, then we manipulate the date portion so that we can ensure they are in the proper order and then merge delisted with crsp to create our total return column, which we will call 'ret'. We then attach the lagged market return and end up with a row in our dataframe for each stock-year-month. The market equity can be found by multiply the number of shares outstanding 'shrout' by price per share 'prc'.

In [20]:
# Clean crsp_raw data
crsp_raw = crsp_raw.sort_values(['permno','date']).reset_index(drop=True).copy()
crsp_raw[['permno', 'permco']] = crsp_raw[['permno', 'permco']].astype(int)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'], format='%y-%m-%d') + MonthEnd(0)
crsp_raw['prc'] = np.absolute(crsp_raw['prc'])

# Clean dlret_raw data
dlret_raw = dlret_raw.sort_values(['permno', 'dlstdt']).reset_index(drop=True).copy()
dlret_raw.permno = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw['date'] = dlret_raw['dlstdt'] + MonthEnd(0)

# Merge crsp_raw with dlret_raw
stocks = crsp_raw.merge(dlret_raw[['permno','date','dlret']], how='outer', on=['permno','date'])
stocks['dlret'] = stocks['dlret'].fillna(0)
stocks['shrcd'] = stocks['shrcd'].ffill()
stocks['exchcd'] = stocks['exchcd'].ffill()
stocks['ret'] = stocks['ret'].fillna(0)
stocks['prc'] = stocks['prc'].fillna(0)
stocks['shrout'] = stocks['shrout'].fillna(0)
stocks['me'] = stocks['prc']*stocks['shrout']

# Redefine 'ret' to include delisted return
stocks['ret'] = (1 + stocks['ret']) * (1 + stocks['dlret']) - 1

# Attach Lagged Market Equity (to be used as weights) for each stock-year-month
stocks = stocks.sort_values(by=['permno','date']).reset_index().drop('index',axis=1).copy()
stocks['daten'] = stocks['date'].dt.year*12 + stocks['date'].dt.month
stocks['IsValidLag'] = stocks['daten'].diff(1) == 1 # Lag date has to be the lagged date
stocks.loc[stocks[stocks['permno'].diff(1) != 0].index,['IsValidLag']] = False # Lagged date has to be the same security
stocks['lme'] = stocks[['permno','me']].groupby('permno').shift(1)
stocks.loc[stocks[stocks['IsValidLag'] == False].index,['lme']] = np.nan
stocks = stocks.drop(['IsValidLag','daten'], axis=1)

# Filter for 'shrcd' = 10 or 11 and 'exchcd' = 1, 2 or 3 per the Fama French website
stocks = stocks[((stocks['shrcd'] == 10) | (stocks['shrcd'] == 11)) & ((stocks['exchcd'] == 1) | (stocks['exchcd'] == 2) | (stocks['exchcd'] == 3))].copy()

# Create year and month columns for readability
stocks['year'] = stocks['date'].dt.year
stocks['month'] = stocks['date'].dt.month

# Rearrange columns into desired format
stocks = stocks[['permno', 'year', 'month', 'exchcd', 'prc', 'shrout', 'lme', 'ret']]

# Drop NAs so that columns without lagged market equity value are not included
stocks = stocks.dropna()

In the below function momentum_ranking, I iterate through every month present in our stock data and find the last 12 months of returns for each stock. Next, I only consider the stocks that existed continuously for the last 13 months (current month plus last 12 months). I do this so we do not have stocks with missing data for this time period that could cause errors in the code because this method is easily replicable in real life. While Daniel and Moskowitz also considered utilizing stocks that were present in month $t-12$ plus another 7 different months plus month $t-1$, I found this to make an extremely small impact on our results. Therefore, we only considered stocks that existed for all of the previous 13 months. I then summed all of the stock returns for months $t-12$ to $t-2$, giving us 11 months of return data. I did not include data for month $t-1$, per direction from Daniel and Moskowitz. I then ranked all stocks, with 1 being the lowest return and a higher ranking corresponding to a higher return. Every month, each stock will get a new ranking, so our output will be for every stock-month-year.

The final output is from 1927-2023, with a row for every year-month-stock. We are just calculating total returns and not factoring in the risk free rate to determine excess returns.

In [21]:
def momentum_ranking(df):
    
    # Initialize final dataframe
    final_df = pd.DataFrame()
    
    # First let's create a list containing our dates, so we can find the last 13 months of data for momentum
    date = df[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
    df['daten'] = df['year']*12 + df['month']
    
    # Iterate through all available year-months for stock data
    for index, row in date.iterrows():
        
        year = row['year']
        month = row['month']
        daten = year*12 + month
        
        # Filter for all stock-year-months from 12 months ago to current year-month
        momentum_stocks = df[(df['daten'] <= daten) & (df['daten'] >= daten - 12)].copy()

        # Next filter for the stocks present in all of the previous 13 months
        counts = momentum_stocks['permno'].value_counts()
        valid_permnos = counts[counts == 13].index
        momentum_stocks = momentum_stocks[momentum_stocks['permno'].isin(valid_permnos)].copy()

        # Divide momentum stock past data (t-12 to t-2) from present data (t) and dropping one month lag (t-1) 
        present = momentum_stocks[momentum_stocks['daten'] == daten]
        past = momentum_stocks[momentum_stocks['daten'] <= daten - 2].copy()
        ret_by_permno = past.groupby('permno')['ret'].sum().reset_index()
        ret_by_permno.columns = ['permno', '11_month_ret']
            
        # Merge the past stock info with present to attach the 11_month_ret column to the momentum stocks
        present = present.merge(ret_by_permno, on='permno', how='inner')
        present['ranking_ret'] = present['11_month_ret'].rank(ascending=True)

        # Reorder columns into desired format
        present = present[['year', 'month', 'permno', 'exchcd', 'lme', 'ret', 'ranking_ret']]
            
        # Add this month data frame to final dataframe
        final_df = pd.concat([final_df, present])
        
    return(final_df)

In [22]:
a1 = momentum_ranking(stocks)

For Daniel and Moskowitz, we have the following information for momentum decile classification: "All firms meeting the data requirements are then placed into one of ten decile portfolios based on this ranking, where portfolio 10 represents the winners (those with the highest past returns) and portfolio 1 the losers." This means the stocks are evenly divided into different deciles. For example, if we have 20 stocks and one stock had the 2nd best returns and another stock had the 10th best returns, we would place them into deciles 1 and 5, respectively. 

The portfolios used in Ken French's (KRF) data library were formed in a manner largely consistent with the decile momentum portfolios in Daniel and Moskowitz. The biggest difference was that the portfolio breakpoints for the KRF portfolios were set so that each of the portfolios has an equal number of NYSE firms (sharecode = 1). This means we filtered for stocks in the NYSE firms and evenly sorted them out into deciles 1 through 10. I used this sorting to define the new boundaries and set the remaining firms according to this boundary. Every month-year-stock has two different decile rankings for both the DM portfolio and the KRF portfolio. These deciles are usually the same for each stock, but could differ by one (or more) deciles for a given month-year-stock. For example, stocks in 1927 were almost entirely NYSE stocks, so the two ranking systems were mostly the same. As time progressed, however, the other exchanges became more prominent and the two decile ranking systems could differ significantly from one another.

One reason we want to divide all stocks based on the NYSE stocks is because the NYSE is a simple measure to include largely traded stocks. If our bottom portfolio contained no NYSE stocks, it could skew our portfolio and increase risk with many low cap, less traded stocks. This method is a good way to ensure the portfolio is more easily constructable.

The final output of our decile_ranking function is from 1927-2023.

In [23]:
def decile_ranking(df):
    
    # Create final_df to store all of our values from each month and year
    final_df = pd.DataFrame()
    
    # First let's create a list containing our dates, so we can find the last 13 months of data for momentum
    date = df[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
    
    # Iterate through all available year-months for stock data
    for index, row in date.iterrows():
        
        year = row['year']
        month = row['month']
            
        # Create dataframe of all stocks for this specific month and date
        all_stocks = df[(df['year'] == year) & (df['month'] == month)].copy()

        # Create dataframe of all NYSE stocks for this specific month and date
        nyse = df[(df['year'] == year) & (df['month'] == month) & (df['exchcd'] == 1)].copy()
        
        # Ensure nyse has enough stocks to sort data
        if (len(nyse) >= 10):
        
            # Get ranges for the Ranking_Ret quartiles
            nyse['quartile'] = pd.qcut(nyse['ranking_ret'], q=10, labels=False)
            quartile_ranges = nyse.groupby('quartile')['ranking_ret'].agg(['min', 'max'])
            
            # Increase max index to unbounded so we do not leave any stocks behind
            quartile_ranges.iloc[9, 1] = np.inf
            quartile_ranges += 1
            stock_bins = [0] + quartile_ranges['max'].tolist()

            # Map ranges to all stocks giving us the KRF_decile
            all_stocks['krf_decile'] = pd.cut(all_stocks['ranking_ret'], bins=stock_bins, labels=range(0, 10), include_lowest=True)
            all_stocks['dm_decile'] = pd.cut(all_stocks['ranking_ret'], bins=10, labels=False)
            
            # Add one to the deciles to match the D&M Paper format
            all_stocks['krf_decile'] = all_stocks['krf_decile'].astype(int) + 1
            all_stocks['dm_decile'] = all_stocks['dm_decile'].astype(int) + 1
            
            # Reorder columns into desired format
            all_stocks = all_stocks[['year', 'month', 'permno', 'lme', 'ret', 'dm_decile', 'krf_decile', 'exchcd']]
            final_df = pd.concat([final_df, all_stocks])
        
    return(final_df)

In [24]:
a2 = decile_ranking(a1)

The below monthly_decile_returns function uses the lagged market cap and returns to find the value-weighted return on each portfolio decile for each year-month. We then end up with 10 different returns for each month of our data. I start by filtering for stocks in a specific month, year and decile. Next, I find the total lagged market cap and then the realized return. I divide the total return by total market cap to find out return for that month. From the output dataframe, we see a high correlation between decile and return, with the higher decile typically corresponding to a higher return. 

Our output is from 1927-2023 and the columns 'dm_ret' and 'krf_ret' show total return and not excess return.

In [25]:
def monthly_decile_returns(stock_ranking, french):
    
    # Create lists for our final dataframe
    years = []
    months = []
    deciles = []
    dm_rets = []
    krf_rets = []
    rfs = []
    
    # Create list of deciles to iterate through
    decile_list = [1,2,3,4,5,6,7,8,9,10]
    
    # Find all unique year-month combinations present in both stocks and french
    momentum_date = stock_ranking[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
    french_date = french[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
    date = french_date.merge(momentum_date, how='inner', on=['year', 'month'])

    # Iterate over the DataFrame and print year and month
    for index, row in date.iterrows():
        # Find monthly returns
        year = row['year']
        month = row['month']
        this_month = stock_ranking[(stock_ranking['year'] == year) & (stock_ranking['month'] == month)].copy()
            
        for i in decile_list:
            # Create decile list for both portfolios
            dm_decile = this_month[this_month['dm_decile'] == i].copy()
            krf_decile = this_month[this_month['krf_decile'] == i].copy()

            # Calculate DM returns for specific Year-Month-Decile
            dm_last_month_value = dm_decile['lme'].sum()
            dm_this_month_value = (dm_decile['lme']*(1 + dm_decile['ret'])).sum()
            dm_ret = (dm_this_month_value - dm_last_month_value)/dm_last_month_value

            # Calculate KRF returns for specific Year-Month_Decile
            krf_last_month_value = krf_decile['lme'].sum()
            krf_this_month_value = (krf_decile['lme']*(1 + krf_decile['ret'])).sum()
            krf_ret = (krf_this_month_value - krf_last_month_value)/krf_last_month_value
            
            # Calculate risk free rates
            rf = french[(french['year'] == year) & (french['month'] ==  month)]['rf'].iloc[0]

            # Append all of our data for our new dataframe
            years.append(year)
            months.append(month)
            deciles.append(i)
            dm_rets.append(dm_ret)
            krf_rets.append(krf_ret)
            rfs.append(rf)
                
    data = {
    'year': years,
    'month': months,
    'decile': deciles,
    'dm_ret': dm_rets,
    'krf_ret': krf_rets,
    'rf': rfs}

    final_df = pd.DataFrame(data)

    return(final_df) 

In [26]:
a3 = monthly_decile_returns(a2, french)

In the below generate_dm_table function, I am recreating the data table shown on the Daniel Moskowitz paper. I utilize our dataframe output from monthly_decile_returns as the input since this contains each year-month-decile plus the risk free rate. We can then filter our dataframe for each specific decile and find the average monthly return. We convert from monthly return to annualized return using the formula below:

$$
r_{annual} = 12*r_{monthly}
$$

And from monthly standard deviation to annualized standard deviation using the formula below:

$$
\sigma_{annual} = \sqrt{12}*\sigma_{monthly}
$$

The sk(m) column is the full-period realized skewness of the monthly log returns (not excess) to the portfolios. We used the below formula for this:

$$
R_t = ln\big(\frac{P_t}{P_{t-1}}\big)
$$

The WML portfolio can be viewed as the zero initial value long-short portfolio that shorts decile 1 and invests that money into decile 10. 

Before we implement the generate_dm_table function, we first need to filter our data frame to achieve the dates in the Daniel and Moskowitz paper. This means we need all year-month-decile data from January 1927 to March 2013. I also import the actual dm returns to find the correlation with our replicated results.

In [27]:
a3 = a3[(a3['year'] >= 1927) & ((a3['year'] < 2013) | ((a3['year'] == 2013) & (a3['month'] <= 3)))].copy()

The below code uses the 'm_m_pt_tot.txt' file containing the actual year-month-deciles for the DM portfolio. The below code imports the data, converts the date to month and year, rearranges the data per the problem input, and then  splices for the desired dates of the Daniel and Moskowitz paper. We need this data to find the correlation between the DM returns and the replicated returns.

In [28]:
# Pull data from our txt file
df_dm_data = pd.read_csv('m_m_pt_tot.txt', header=None, sep='\s+', names=['date', 'decile', 'DM_Ret_actual', 'd', 'e'])

# Convert 'date' column to month and year
df_dm_data['year'] = df_dm_data['date'].apply(lambda x: int(str(x)[:4]))
df_dm_data['month'] = df_dm_data['date'].apply(lambda x: int(str(x)[4:6]))
df_dm_data = df_dm_data.rename(columns = {'DM_Ret_actual': 'dm_ret_actual'})

# Rearrange columns
df_dm_data = df_dm_data[['year', 'month', 'decile', 'dm_ret_actual']]

In [29]:
def generate_dm_table(decile_returns, dm_actual):
    
    # Merge df1 and df2 for simplicity for each year-month-decile
    df = pd.merge(decile_returns, dm_actual, how = 'inner', on = ['year', 'month', 'decile'])
    decile_returns = []
    sds = []
    SRs = []
    sk_ms = []
    correlations = []
    
    # Iterate through the 10 different deciles
    for i in range(1,11):
        
        # Calculate average annual excess return
        this_decile_return = 1200*(df[df['decile'] == i]['dm_ret'] - df[df['decile'] == i]['rf']).mean()
        
        # Calculate average annual standard deviation
        sd = 100*np.sqrt(12)*(df[df['decile'] == i]['dm_ret'] - df[df['decile'] == i]['rf']).std()
        
        # Calculate average annual Sharpe Ratio
        SR = this_decile_return/sd
        
        # Calculate average sk(m)
        sk_m = (np.log(df[df['decile'] == i]['dm_ret'] + 1)).skew()
        
        # Calculate correlation between replicated and actual datasets
        correlation = df[df['decile'] == i]['dm_ret'].corr(df[df['decile'] == i]['dm_ret_actual'])
        
        # Append our results to each respective list
        decile_returns.append(this_decile_return)
        sds.append(sd)
        SRs.append(SR)
        sk_ms.append(sk_m)
        correlations.append(correlation)

    # Compute WML metrics, using same methods as above
    W = df[df['decile'] == 10]['dm_ret'].reset_index(drop=True)
    L = df[df['decile'] == 1]['dm_ret'].reset_index(drop=True)
    WML_ret = 1200*(W - L).mean()
    WML_std = 100*np.sqrt(12)*(W - L).std()
    WML_SR = WML_ret/WML_std
    WML_skm = (np.log(W - L + 1)).skew()
    WML_corr = (W - L).reset_index(drop=True).corr(df[df['decile'] == 10]['dm_ret_actual'].reset_index(drop=True) - df[df['decile'] == 1]['dm_ret_actual'].reset_index(drop=True))

    # Append WML to lists
    decile_returns.append(WML_ret)
    sds.append(WML_std)
    SRs.append(WML_SR)
    sk_ms.append(WML_skm)
    correlations.append(WML_corr)
    
    # Construct our new data frame as our output
    data = {
    'deciles': ['decile 1', 'decile 2', 'decile 3', 'decile 4', 'decile 5', 'decile 6', 'decile 7', 'decile 8', 'decile 9', 'decile 10', 'wml'],
    '$r - r_f$': decile_returns,
    'σ': sds,
    'SR': SRs,
    'sk(m)': sk_ms,
    'correlations': correlations
    }


    final_df = pd.DataFrame(data)
    return(final_df.round(2).T)

In the below table, we can see our replicated Daniel and Moskowitz data for January 1926 to March 2013. While that data is not an exact replication and can be off by as much as a percent for some of the deciles, the general trends in the replication portfolio are extremely similar to the paper. We can see that we typically have a strong correlation of around 95\% to 97\%. We also see extremely strong trends and a general increase as we move from decile 1 to decile 10. The WML long-short portfolio return was found to be 17.23\%, showing the "magic" of the momentum portfolio. 

One of the reasons the correlations are not perfect would be because I required stocks to be consistently listed for 13 months straight. Some of the Daniel and Moskowitz stocks could have only existied for 8 months out of the 13 months and still be included in the DM portfolio. I thought my method was easier to implement, more realistic and would not bring up several NaN erros in the code. Plus with my method, we gain a more comprehensive picture of the stock momentum directions.

Another possible reason for not obtaining exact replication is my choosing stocks that only exist on the NYSE, AMEX and NASDAQ. Daniel and Moskowitz could have considered stocks from other exchanges or countries. There is also a possibility of them using other methods unspecified in their paper when constructing these momentum portfolios. Regardless of not obtaining the exact numbers, we still achieved a very high correlation that shows the power of a momentum portfolio.

In [30]:
a4 = generate_dm_table(a3, df_dm_data)
print('    Average Annual DM Momentum Portfolio metrics from January 1927 to March 2013')
a4

    Average Annual DM Momentum Portfolio metrics from January 1927 to March 2013


,0,1,2,3,4,5,6,7,8,9,10
deciles,decile 1,decile 2,decile 3,decile 4,decile 5,decile 6,decile 7,decile 8,decile 9,decile 10,wml
$r - r_f$,-1.23,2.56,3.84,6.26,7.49,7.41,9.38,10.68,11.82,15.59,16.83
σ,33.61,26.96,23.6,21.41,20.5,19.53,19.7,20.46,21.69,26.76,28.28
SR,-0.04,0.09,0.16,0.29,0.37,0.38,0.48,0.52,0.54,0.58,0.6
sk(m),-0.15,-0.1,-0.04,-0.12,-0.29,-0.53,-0.53,-0.45,-0.58,-0.88,-5.49
correlations,0.97,0.97,0.97,0.96,0.97,0.96,0.97,0.96,0.97,0.97,0.91


The below generate_krf_table function will do the same as the generate_dm_table function except it uses the 'krf_ret' returns instead of the dm_ret. Since the KRF returns utilized a different method from what Daniel and Moskowitz used to construct their portfolio, we observe it to have a lower correlation than our DM replication.

In [31]:
def generate_krf_table(decile_returns, dm_actual):
    
    # Merge df1 and df2 for simplicity
    df = pd.merge(decile_returns, dm_actual, how = 'inner', on =['year', 'month', 'decile'])
    decile_returns = []
    sds = []
    SRs = []
    sk_ms = []
    correlations = []
    for i in range(1,11):
        this_decile_return = 1200*(df[df['decile'] == i]['krf_ret'] - df[df['decile'] == i]['rf']).mean()
        sd = 100*np.sqrt(12)*(df[df['decile'] == i]['krf_ret'] - df[df['decile'] == i]['rf']).std()
        SR = this_decile_return/sd
        sk_m = (np.log(df[df['decile'] == i]['krf_ret'] + 1)).skew()
        correlation = df[df['decile'] == i]['krf_ret'].corr(df[df['decile'] == i]['dm_ret_actual'])
        # Append our results to each respective list
        decile_returns.append(this_decile_return)
        sds.append(sd)
        SRs.append(SR)
        sk_ms.append(sk_m)
        correlations.append(correlation)

    # Compute WML metrics
    W = df[df['decile'] == 10]['krf_ret'].reset_index(drop=True)
    L = df[df['decile'] == 1]['krf_ret'].reset_index(drop=True)
    WML_ret = 1200*(W - L).mean()
    WML_std = 100*np.sqrt(12)*(W - L).std()
    WML_SR = WML_ret/WML_std
    WML_skm = (np.log(W - L + 1)).skew()
    WML_corr = (W - L).reset_index(drop=True).corr(df[df['decile'] == 10]['dm_ret_actual'].reset_index(drop=True) - df[df['decile'] == 1]['dm_ret_actual'].reset_index(drop=True))

    # Append WML to lists
    decile_returns.append(WML_ret)
    sds.append(WML_std)
    SRs.append(WML_SR)
    sk_ms.append(WML_skm)
    correlations.append(WML_corr)
    
    data = {
    'deciles': ['decile 1', 'decile 2', 'decile 3', 'decile 4', 'decile 5', 'decile 6', 'decile 7', 'decile 8', 'decile 9', 'decile 10', 'wml'],
    '$r - r_f$': decile_returns,
    'σ': sds,
    'SR': SRs,
    'sk(m)': sk_ms,
    'correlations': correlations
    }

    final_df = pd.DataFrame(data)
    return(final_df.round(2).T)

From the table below, we can see we do not achieve as high of a WML return compared to the Daniel and Moskowitz portfolio construction. We should note, however, that this WML has a smaller annual standard deviation (around 3\% less). This makes intuitive sense since we could consider the NYSE stocks to be larger cap and slightly more stable. The means the KRF portfolio is less likely to have many lower cap, higher risk, non NYSE stocks to compose our loser portion of the portfolio, making the KRF decile 1 less risky. So while the KRF portfolio does not achieve returns as high as the DM portfolio, it likely performs better than the DM portfolio during market downturns. 

In [32]:
a5 = generate_krf_table(a3, df_dm_data)
print('    Average Annual KRF Momentum Portfolio metrics from January 1927 to March 2013')
a5

    Average Annual KRF Momentum Portfolio metrics from January 1927 to March 2013


,0,1,2,3,4,5,6,7,8,9,10
deciles,decile 1,decile 2,decile 3,decile 4,decile 5,decile 6,decile 7,decile 8,decile 9,decile 10,wml
$r - r_f$,0.69,4.86,5.32,6.65,7.02,7.71,8.85,10.45,11.03,14.75,14.06
σ,31.26,25.83,22.68,20.85,20.22,19.57,19.33,19.98,20.64,25.04,25.4
SR,0.02,0.19,0.23,0.32,0.35,0.39,0.46,0.52,0.53,0.59,0.55
sk(m),0.02,-0.08,0.02,-0.06,-0.32,-0.55,-0.48,-0.51,-0.58,-1.04,-5.57
correlations,0.96,0.95,0.95,0.94,0.95,0.96,0.96,0.96,0.97,0.97,0.88


## Would you implement this trading strategy if you were running your own fund? What are the main implementation challenges to consider?

While the momentum trading strategies like the DM portfolio we constructed above tend to produce returns well in excess of the average market return, we have to be aware of this strategy's risks before we invest in such a portfolio. One such risk is a high standard deviation as we saw on our final tables. The DM annualized standard deviation was around 28\%, which is slightly higher than the average market annualized standard deviation. We also saw from the graph above the higher potential for bad returns when compared to an unleveraged portfolio. This contributes to a much higher value at risk than other portfolios such as the 60-40 stock bond portfolio we constructed earlier. 

Personally, I would implement this WML portfolio as a trading strategy since the exceptional returns seem to outweigh the potentially large downsides. Before adopting this trading method, however, I would spend considerable time studying historical data to determine trends in momentum. One such trend that is likely to appear is the WML portfolio relationship to the VIX index, with a higher VIX score corresponding to higher losses on the WML portfolio, on average. This is due to the momentum portfolio failing during market downturns. 

Considering just data from 1963 to 2023, the worst months during this time period for the momentum portfolios were:

1.	April 2009:	-34.3%	

2.	January 2001:	-25.3%	

3.	November 2002:	-16.3%	

4.	January 2023:	-16.0%

5.	January 1975:	-13.8%

6.	May 2009:       -12.5%	

7.	November 2020:	-12.4%	


Using the historical value at risk method, this corresponds to a 99% VaR of -12.4% for our WML portfolio. We would therefore need to be comfortable with this VaR number before proceeding with the WML method. If we were investing client's money, they would need to be aware of this strategy's upsides and risks.

Other strategies to implement would be avoiding using the momentum strategy during market downturns. We can see in the list above that this strategy did poorly during the 2009 crash as well as the internet bubble. If we can find a way to not use this strategy during these periods of high volatility, we have a better chance of achieving good returns for the WML portfolio and avoiding the months with huge losses.

Other things to consider for this portfolio is if the WML will continue to work into the future. It typically does well when growth stocks do well. Lately the growth stocks have been mainly driven by tech. If tech begins to do poorly and value stocks tend to do better, we could see some sort of mean reversion. In the case of a mean reversion, this portfolio will perform poorly. 

To sum up, the WML strategy is a remarkable "magic" strategy that achieves returns higher on average than the market returns. It comes with its own risks and tends to have very bad months that can result in huge losses. We need to be fully aware of the risks and proceed with caution before pursing this trading strategy. If implemented for a long enough time, the returns tend to be very good.